In [ ]:
!pip install hyperas

In [2]:
import h5py
from os.path import join,exists
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.optimizers import RMSprop
from hyperas.distributions import choice
from hyperas import optim
from keras.callbacks import ModelCheckpoint
from keras.constraints import maxnorm
from random import randint
from keras import backend as K
import os
import numpy as np
from hyperopt import Trials, STATUS_OK, tpe
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve,roc_auc_score
K.set_image_data_format('channels_first')

In [3]:
os.chdir('/content/drive/My Drive/Colab Notebooks/synopsys-2022/models/classification')

In [4]:
def create_model(X_train, Y_train, X_test, Y_test):
    W_maxnorm = 3
    DROPOUT = {{choice([0.3,0.5,0.7])}}
    # DROPOUT=0.3

    model = Sequential()
    model.add(Conv2D(64, (1, 5), padding='same', input_shape=(20, 1, 20),activation='relu',kernel_constraint=maxnorm(W_maxnorm)))
    model.add(MaxPool2D(pool_size=(1, 2),strides=(1,2)))
    model.add(Flatten())

    model.add(Dense(16,activation='relu',kernel_constraint=maxnorm(W_maxnorm)))
    model.add(Dropout(DROPOUT))
    model.add(Dense(2,kernel_constraint=maxnorm(W_maxnorm),activation='softmax'))


    myoptimizer = RMSprop(learning_rate={{choice([1e-1,0.01,0.001,0.0001,1e-5])}}, rho=0.9, epsilon=1e-06)
    mylossfunc = 'binary_crossentropy'
    model.compile(loss=mylossfunc, optimizer=myoptimizer, metrics=['accuracy'])
    # model.compile(loss=RMSprop, optimizer=myoptimizer,metrics=['accuracy'])
    result = model.fit(X_train, Y_train, batch_size=100, epochs=5,validation_split=0.1,verbose=False)

    val_loss = np.amax(result.history['val_loss'])

    # score, acc = model.evaluate(X_test,Y_test, verbose=False)
    # model_arch = 'MODEL_ARCH'
    # bestaccfile = join('TOPDIR',model_arch,model_arch+'_hyperbestacc')
    # reportAcc(acc,score,bestaccfile)

    return {'loss': val_loss, 'status': STATUS_OK,'model':model}

In [5]:
def train_data():
  data_train = h5py.File('./data/train.h5.batch1','r')
  X_train = np.array(data_train['data'])
  Y_train = np.array(data_train['label'])
  X_train_shuffled, Y_train_shuffled = shuffle(X_train, Y_train)

  data_test = h5py.File('./data/test.h5.batch1', 'r')
  X_test = np.array(data_test['data'])
  Y_test = np.array(data_test['label'])
  X_test_shuffled, Y_test_shuffled = shuffle(X_test, Y_test)
  return X_train_shuffled, Y_train_shuffled, X_test_shuffled, np.fliplr(Y_test_shuffled)

In [7]:
best_run, best_model = optim.minimize(model=create_model,
                                          data=train_data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='seq_64x1_16',
                                          verbose=False)

100%|██████████| 10/10 [01:57<00:00, 11.71s/it, best loss: 0.15054036676883698]


In [8]:
X_train, Y_train, X_test, Y_test = train_data()
best_model.fit(X_train, Y_train, batch_size=100, epochs=20,validation_split=(0.1))

Epoch 1/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2762 - accuracy: 0.9173 - val_loss: 0.2396 - val_accuracy: 0.9322
Epoch 2/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2673 - accuracy: 0.9198 - val_loss: 0.2326 - val_accuracy: 0.9347
Epoch 3/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2591 - accuracy: 0.9236 - val_loss: 0.2264 - val_accuracy: 0.9382
Epoch 4/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2541 - accuracy: 0.9251 - val_loss: 0.2206 - val_accuracy: 0.9407
Epoch 5/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2491 - accuracy: 0.9278 - val_loss: 0.2166 - val_accuracy: 0.9434
Epoch 6/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2438 - accuracy: 0.9274 - val_loss: 0.2134 - val_accuracy: 0.9442
Epoch 7/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2389 - accuracy: 0.9306 - val_loss: 0.2095 - val_accuracy: 0.9453
Epoch 

In [9]:
y_pred = best_model.predict(X_test)
roc_auc_score(Y_test,y_pred)

0.971504402195287

In [11]:
best_model.save('./seq_64x1_16')

INFO:tensorflow:Assets written to: ./seq_64x1_16/assets
